<a href="https://colab.research.google.com/github/sungjin417/sesac-phython/blob/master/%EA%B5%90)%EB%8C%80%EA%B7%9C%EB%AA%A8_LLM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%A7%80%EC%8B%9D_%EC%B1%97%EB%B4%87_%EA%B0%9C%EB%B0%9C_2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🌼 대규모 LLM을 활용한 지식 챗봇 개발 - 2차시(24.11.22)

---


## 필요 패키지 설치

In [2]:
!pip install transformers

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
!pip install accelerate

In [5]:
!pip install sentencepiece

In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [7]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.5 MB/s eta 0:00:00


## Data Set

In [8]:
from google.colab import drive
drive.mount('/content/drive')
# 모두 체크하고 동의 하기

Mounted at /content/drive


In [9]:
import pandas as pd
df=pd.read_excel('/content/drive/MyDrive/transformers_data.xlsx')

In [10]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
train_df, temp_df = train_test_split(df, test_size = 0.1, random_state = 42)
valid_df, test_df = train_test_split(temp_df, test_size = 0.1, random_state=42)
# train : 90%, vaild + test : 10%
# valid : 90%, test : 10%

In [12]:
# 허깅 페이스 데이터 셋으로 바꿔줘야 함
train_dataset = Dataset.from_pandas(train_df,preserve_index = False)
valid_dataset = Dataset.from_pandas(valid_df,preserve_index = False)
test_dataset = Dataset.from_pandas(test_df,preserve_index = False)

In [13]:
dataset = DatasetDict({
    'train' : train_dataset,
    'valid' : valid_dataset,
    'test' : test_dataset
})

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 81000
    })
    valid: Dataset({
        features: ['en', 'ko'],
        num_rows: 8100
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 900
    })
})

In [15]:
print(dataset['train']['en'][:3])

["This time we'd like to get an invoice to export the equipment to Vietnam.", 'Then is the bag strap and floor made of leather?', "Are you badly hurt, don't worry about work and get a good rest."]


In [16]:
print(dataset['train']['ko'][:3])

['이번에 베트남으로 장비를 수출하기 위해 송장을 발급받으려고 합니다.', '가방 스트랩이랑 바닥은 가죽 소재인가요?', '많이 다치셨나요, 회사는 걱정하지 마시고 푹 쉬다 오셔요.']


In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq # 앞쪽에 공백을 주는 모델
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import torch
import multiprocessing

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [19]:
model_ckpt = "KETI-AIR/ke-t5-base" # 모델의 체크 포인트
max_token_length = 64

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
# 사전 학습된 모델의 토크나이저 불러오기

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [21]:
dataset['train'][5]['en']

'Then I will bring the pork belly first, please be careful as the grill is hot.'

In [22]:
dataset['train'][5]['ko']

'그럼 삼겹살 먼저 올려드릴게요, 불판 뜨거우니 조심하세요.'

In [23]:
tokenized_sample_en = tokenizer(dataset['train'][5]['en'],
                                max_length = max_token_length,
                                padding = True, truncation = True)
tokenized_sample_en
# input_ids : 입력텍스트를 토크화하여 정수 인덱스로 변환
# attention_mask : 패딩과 실제 데이터를 구분하는 마스킹
# 1 : 실제 데이터, 0 : 패딩

{'input_ids': [5298, 83, 96, 2611, 5, 40651, 67, 14731, 275, 4, 11018, 67, 25818, 64, 5, 43509, 43, 6327, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
tokenized_sample_ko = tokenizer(dataset['train'][5]['ko'],
                                max_length = max_token_length,
                                padding = True, truncation = True)
tokenized_sample_ko

{'input_ids': [5787, 30716, 757, 4434, 45550, 4, 1101, 1176, 11706, 923, 363, 434, 11623, 14316, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
tokenizer(dataset['train'][:3]['en'],
          max_length = max_token_length,
          padding = True, truncation = True)

{'input_ids': [[465, 248, 146, 17, 136, 297, 10, 403, 93, 20, 52217, 10, 16392, 5, 7387, 10, 17854, 3, 1], [5298, 43, 5, 13676, 61135, 13, 5964, 565, 14, 37525, 85, 1, 0, 0, 0, 0, 0, 0, 0], [9228, 106, 29486, 10470, 4, 697, 17, 71, 14069, 155, 509, 13, 403, 16, 667, 3318, 3, 1, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]}

In [26]:
dataset['train'][:3]['en']

["This time we'd like to get an invoice to export the equipment to Vietnam.",
 'Then is the bag strap and floor made of leather?',
 "Are you badly hurt, don't worry about work and get a good rest."]

In [27]:
pd.DataFrame(
    [
    tokenized_sample_en['input_ids'],
    tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ],
    index = ('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ids,5298,83,96,2611,5,40651,67,14731,275,4,11018,67,25818,64,5,43509,43,6327,3,1
tokens,▁Then,▁I,▁will,▁bring,▁the,▁pork,▁be,lly,▁first,",",▁please,▁be,▁careful,▁as,▁the,▁grill,▁is,▁hot,.,</s>


In [28]:
pd.DataFrame(
    [
    tokenized_sample_ko['input_ids'],
    tokenizer.convert_ids_to_tokens(tokenized_sample_ko['input_ids'])
    ],
    index = ('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
ids,5787,30716,757,4434,45550,4,1101,1176,11706,923,363,434,11623,14316,3,1
tokens,▁그럼,▁삼겹살,▁먼저,▁올려,드릴게요,",",▁불,판,▁뜨,거,우,니,▁조심,하세요,.,</s>


In [29]:
def convert_to_features(text):
  model_inputs = tokenizer(text['en'],
                           text_target = text['ko'],
                           max_length = max_token_length,
                           truncation = True
                           )
  return model_inputs
  # en, ko, input_ids, attention_mask, labels -> en과 ko는 제거(나중에 오류 발생 가능)
  # input_ids : 영어 데이터의 벡터화
  # attention_mask : 패딩유무
  # labels : 한국어 데이터의 벡터화

In [30]:
tokenized_dataset = dataset.map(convert_to_features,
                                batched = True,
                                remove_columns = dataset['train'].column_names)

Map:   0%|          | 0/81000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

In [31]:
tokenized_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 81000
})

In [32]:
# 데이터 프레임으로 찍어보기
tokenized_dataset['train'].to_pandas().head()
# Q. attention_mask는 영어와 한국어중에 어떤 데이터를 기준으로 씌어진 패딩일까?
# -> 영어 기준

,input_ids,attention_mask,labels
0,"[465, 248, 146, 17, 136, 297, 10, 403, 93, 20,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2172, 2043, 45, 15227, 1030, 288, 105, 865, 1..."
1,"[5298, 43, 5, 13676, 61135, 13, 5964, 565, 14,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[11845, 63140, 13093, 8550, 19, 17299, 2457, 2..."
2,"[9228, 106, 29486, 10470, 4, 697, 17, 71, 1406...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[354, 28531, 54441, 4, 7983, 38241, 14173, 162..."
3,"[523, 84, 4021, 106, 93, 5293, 54, 5, 4966, 4,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[42907, 2676, 31502, 1043, 5831, 6265, 757, 14..."
4,"[55, 2228, 96, 624, 993, 703, 16, 19587, 65, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[19486, 19, 26289, 5779, 41, 1136, 7268, 1560,..."


In [33]:
print('원본 데이터', dataset['train'][5]['en'])
print('처리 후 데이터', tokenized_dataset['train'][5]['input_ids'])
print('토킅화된 데이터', tokenizer.convert_ids_to_tokens(tokenized_dataset['train'][5]['input_ids']))

원본 데이터 Then I will bring the pork belly first, please be careful as the grill is hot.
처리 후 데이터 [5298, 83, 96, 2611, 5, 40651, 67, 14731, 275, 4, 11018, 67, 25818, 64, 5, 43509, 43, 6327, 3, 1]
토킅화된 데이터 ['▁Then', '▁I', '▁will', '▁bring', '▁the', '▁pork', '▁be', 'lly', '▁first', ',', '▁please', '▁be', '▁careful', '▁as', '▁the', '▁grill', '▁is', '▁hot', '.', '</s>']


In [34]:
print('원본 데이터', dataset['train'][5]['ko'])
print('처리 후 데이터', tokenized_dataset['train'][5]['labels'])
print('토킅화된 데이터', tokenizer.convert_ids_to_tokens(tokenized_dataset['train'][5]['labels']))

원본 데이터 그럼 삼겹살 먼저 올려드릴게요, 불판 뜨거우니 조심하세요.
처리 후 데이터 [5787, 30716, 757, 4434, 45550, 4, 1101, 1176, 11706, 923, 363, 434, 11623, 14316, 3, 1]
토킅화된 데이터 ['▁그럼', '▁삼겹살', '▁먼저', '▁올려', '드릴게요', ',', '▁불', '판', '▁뜨', '거', '우', '니', '▁조심', '하세요', '.', '</s>']


In [35]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [36]:
encoder_inputs = tokenizer(
    ["Studies have been shown that owning a dog is good for you"],
    return_tensors="pt" # 파이토치텐서형태로 저장
)['input_ids'].to(device)

In [37]:
decoder_targets = tokenizer(
    ["개를 키우는 것이 건강에 좋다는 연구 결과가 있습니다."],
    return_tensors="pt"
)['input_ids'].to(device)

In [38]:
print(encoder_inputs)

tensor([[24611,    84,   166,  8135,    38,   847,    91,    16,  8146,    43,
           667,    40,   106,     1]], device='cuda:0')


In [39]:
print(decoder_targets)

tensor([[15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3,     1]],
       device='cuda:0')


In [40]:
decoder_inputs = model._shift_right(decoder_targets)
# 시퀀스를 오른쪽으로 한 칸 이동시켜서 디코더에 입력 시작 신호 토큰을 넣어줌

In [41]:
decoder_inputs

tensor([[    0, 15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3]],
       device='cuda:0')

In [42]:
decoder_targets

tensor([[15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3,     1]],
       device='cuda:0')

In [43]:
pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index = ('decoder target', 'decoder input')
)

,0,1,2,3,4,5,6,7,8,9
decoder target,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.,</s>
decoder input,<pad>,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.


In [44]:
outputs = model(input_ids = encoder_inputs,
                decoder_input_ids = decoder_inputs,
                labels = decoder_targets)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [45]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [46]:
outputs.loss # grad_fn=<NllLossBackward0>) : 역전파를 통해 생성해 냈다

tensor(87.8185, device='cuda:0', grad_fn=<NllLossBackward0>)

In [47]:
outputs['encoder_last_hidden_state'].shape
# 데이터 수, 스텝 수, 모델 사이즈
# 인코더에 들어간 1개의 데이터가
# 14개의 토큰을 갖고
# 각각 768차원 벡터로 인코딩 되었다

torch.Size([1, 14, 768])

In [48]:
outputs['logits'].shape
# 디코더에 넣은 데이터 개수 1개
# 그 데이터의 토큰 10개
# -> 다음 단어에 대한 10개의 예측
# 64128개의 단어에 대한 확률값

torch.Size([1, 10, 64128])

In [49]:
tokenizer.convert_ids_to_tokens(torch.argmax(outputs['logits'][0], axis = 1).cpu().numpy()) # .cpu().numpy() : numpy는 cpu에서 작동

['큐브', '큐브', '▁비일비재', '▁비일비재', '▁베네', '▁비일비재', '▁베네', '▁베네', '큐브', '큐브']

In [50]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [51]:
tokenized_dataset['train'][:3]
# 데이터 여러개가 하나로 묶여서 출력
# collator에는 잘라서 넣어줘야 함

{'input_ids': [[465,
   248,
   146,
   17,
   136,
   297,
   10,
   403,
   93,
   20,
   52217,
   10,
   16392,
   5,
   7387,
   10,
   17854,
   3,
   1],
  [5298, 43, 5, 13676, 61135, 13, 5964, 565, 14, 37525, 85, 1],
  [9228,
   106,
   29486,
   10470,
   4,
   697,
   17,
   71,
   14069,
   155,
   509,
   13,
   403,
   16,
   667,
   3318,
   3,
   1]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[2172,
   2043,
   45,
   15227,
   1030,
   288,
   105,
   865,
   1616,
   38230,
   22774,
   1279,
   3,
   1],
  [11845, 63140, 13093, 8550, 19, 17299, 2457, 24409, 85, 1],
  [354,
   28531,
   54441,
   4,
   7983,
   38241,
   14173,
   16275,
   7538,
   66,
   382,
   8689,
   328,
   3,
   1]]}

In [52]:
result = []
for i in range(1, 4):
  result.append(tokenized_dataset['train'][i])
result

[{'input_ids': [5298, 43, 5, 13676, 61135, 13, 5964, 565, 14, 37525, 85, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [11845, 63140, 13093, 8550, 19, 17299, 2457, 24409, 85, 1]},
 {'input_ids': [9228,
   106,
   29486,
   10470,
   4,
   697,
   17,
   71,
   14069,
   155,
   509,
   13,
   403,
   16,
   667,
   3318,
   3,
   1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [354,
   28531,
   54441,
   4,
   7983,
   38241,
   14173,
   16275,
   7538,
   66,
   382,
   8689,
   328,
   3,
   1]},
 {'input_ids': [523,
   84,
   4021,
   106,
   93,
   5293,
   54,
   5,
   4966,
   4,
   262,
   11018,
   4733,
   275,
   3,
   1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [42907, 2676, 31502, 1043, 5831, 6265, 757, 14174, 27138, 3, 1]}]

In [53]:
batch = data_collator(result)
# data_Collator로 패딩. shift_right를 진행

In [54]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [55]:
batch
# -100은 손실함수에서 계산되지 않도록 처리

{'input_ids': tensor([[ 5298,    43,     5, 13676, 61135,    13,  5964,   565,    14, 37525,
            85,     1,     0,     0,     0,     0,     0,     0],
        [ 9228,   106, 29486, 10470,     4,   697,    17,    71, 14069,   155,
           509,    13,   403,    16,   667,  3318,     3,     1],
        [  523,    84,  4021,   106,    93,  5293,    54,     5,  4966,     4,
           262, 11018,  4733,   275,     3,     1,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]), 'labels': tensor([[11845, 63140, 13093,  8550,    19, 17299,  2457, 24409,    85,     1,
          -100,  -100,  -100,  -100,  -100],
        [  354, 28531, 54441,     4,  7983, 38241, 14173, 16275,  7538,    66,
           382,  8689,   328,     3,     1],
        [42907,  2676, 31502,  1043,  5831,  6265,   757, 14174, 27138,     3,


{'input_ids':
tensor([[ 5298,    43,     5, 13676, 61135,    13,  5964,   565,    14, 37525,
            85,     1,     0,     0,     0,     0,     0,     0],
        [ 9228,   106, 29486, 10470,     4,   697,    17,    71, 14069,   155,
           509,    13,   403,    16,   667,  3318,     3,     1],
        [  523,    84,  4021,   106,    93,  5293,    54,     5,  4966,     4,
           262, 11018,  4733,   275,     3,     1,     0,     0]]), 'attention_mask':
           tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]),
        'labels':
        tensor([[11845, 63140, 13093,  8550,    19, 17299,  2457, 24409,    85,     1,
          -100,  -100,  -100,  -100,  -100],
        [  354, 28531, 54441,     4,  7983, 38241, 14173, 16275,  7538,    66,
           382,  8689,   328,     3,     1],
        [42907,  2676, 31502,  1043,  5831,  6265,   757, 14174, 27138,     3,
             1,  -100,  -100,  -100,  -100]]),
              'decoder_input_ids':
               tensor([[    0, 11845, 63140, 13093,  8550,    19, 17299,  2457, 24409,    85,
             1,     0,     0,     0,     0],
        [    0,   354, 28531, 54441,     4,  7983, 38241, 14173, 16275,  7538,
            66,   382,  8689,   328,     3],
        [    0, 42907,  2676, 31502,  1043,  5831,  6265,   757, 14174, 27138,
             3,     1,     0,     0,     0]])}

In [56]:
import evaluate

metric = evaluate.load("sacrebleu")

In [62]:
predict = [
    "저는 딥러닝을 좋아해요.",
    "요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다."
]

answer = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predict, references=answer)

{'score': 100.00000000000004,
 'counts': [21, 19, 17, 15],
 'totals': [21, 19, 17, 15],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 21,
 'ref_len': 21}

In [63]:
predict = [
    "저는 딥러닝을 좋아해요.",
    "딥러닝 프레임워크가 잘 개발되었기 때문에 요즘은 누군가의 도움 없이 기계번역 시스템을 구축할 수 있다."
]

answer = [
    ["저는 딥러닝을 좋아해요.", "나는 딥러닝을 사랑해요."],
    ["요즘은 딥러닝 프레임워크가 잘 발달되어 있기 때문에 누구의 도움 없이도 기계 번역 시스템을 구축할 수 있습니다.",
     "최근에는 딥러닝 프레임워크가 잘 개발되어 있기 때문에 다른 사람의 도움 없이도 기계 번역 시스템을 개발할 수 있습니다."]
]
metric.compute(predictions=predict, references=answer)

{'score': 25.28116160010779,
 'counts': [14, 7, 4, 1],
 'totals': [19, 17, 15, 13],
 'precisions': [73.6842105263158,
  41.1764705882353,
  26.666666666666668,
  7.6923076923076925],
 'bp': 0.9000876262522591,
 'sys_len': 19,
 'ref_len': 21}

[참조] 아래 함수는 모델의 예측과 레이블을 가지고 bleu를 계산하는 헬퍼 함수로  
[트랜스포머 학습 코스 번역기 매트릭](https://huggingface.co/course/chapter7/4?fw=pt#metrics)에서 제공하는 코드!  

In [64]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # 모델의 예측값인 preds와 실제 타겟값인 labels를 분리

    if isinstance(preds, tuple):
      # preds가 튜플이면 첫번째(0)값을 가져온다(loss값)
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # 정수 인덱스 형태의 preds(배치값)를 텍스트로 변환
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # -100을 패딩 토큰으로 대체 (loss값 계산에서 무시하기 위해)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # 라벨값의 인덱스를 텍스트로 변환
    decoded_preds = [pred.strip() for pred in decoded_preds]
    # 예측 문자열의 앞 뒤 공백을 제거
    decoded_labels = [[label.strip()] for label in decoded_labels]
    # 라벨데이터도 동일하게 앞 뒤 공백 제거
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    # BLEU 지표를 계산하는 구간
    result = {"bleu": result["score"]}

    return result

In [67]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chkpt", # 체크포인트 위치
    learning_rate=0.0005, # 가중치
    weight_decay=0.01, # 모델의 과적합 방지에 사용
    per_device_train_batch_size=12, # GPU 한개 당 배치사이즈
    per_device_eval_batch_size=128,
    num_train_epochs=1, # 에포크
    save_steps=500, # 체크포인트
    save_total_limit=2,
    evaluation_strategy="epoch", # 평가를 언제 할건지
    logging_strategy="no", # 로그를 기록한 시점
    predict_with_generate=True, # 평가 시 디코더를 사용해서 출력 시퀀스를 생성 True
    fp16=False, #
    gradient_accumulation_steps=2,
    report_to="none" # Wandb 로그 끄기
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [69]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-69-ad794d612675>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [70]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Bleu
1,No log,3.030064,5.100076


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=3375, training_loss=4.505924479166667, metrics={'train_runtime': 1723.1697, 'train_samples_per_second': 47.006, 'train_steps_per_second': 1.959, 'total_flos': 2501732223959040.0, 'train_loss': 4.505924479166667, 'epoch': 1.0})

In [71]:
trainer.save_model("./results")

In [72]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu()

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


T5ForConditionalGeneration(
  (shared): Embedding(64128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [73]:
# 모델 테스트용 문장
input_text = [
    "Because deep learning frameworks are well developed, in these days, machine translation system can be built without anyone's help.",
    "This system was made by using HuggingFace's T5 model for a one day"
]


In [76]:
inputs = tokenizer(input_text, return_tensors='pt', padding = True, max_length = max_token_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2852: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [77]:
inputs

{'input_ids': tensor([[ 8127,  5859,  5789, 22309,     8,    69,   484,  6560,     4,    20,
           572,  1258,     4,  9872, 46301,  1076,   147,    67,  3807,  1215,
          3993,    17,     8,   787,     3,     1],
        [  465,  1076,    62,   565,    81,  1676,   992, 60049,  1044, 17400,
            17,     8,   745,   466,  3900,    40,    16,   165,   688,     1,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0]])}

In [79]:
koreans = model.generate(
    **inputs,
    # 딕셔너리에 담긴 input_ids, attention_mask를 개별 변수로 전달하는 언패킹
    max_length = max_token_length,
    num_beams = 5
    # beams search : 텍스트 생성의 품질은 높이기 위한 알고리즘
    # 탐색 진행 과정
    # 1. 토큰 생성 시, 가장 높은 확률 or 점수를 가진 5개의 후보 선택
    # 2. 다시 토큰 생성 시, 각 후보들의 경로에서 다시 5개의 후보를 선택
    # 3. 25개의 경로를 유지하면서 계속 반복 후 최종적으로 가장 높은 점수의 토큰을 선택

)
koreans.shape

torch.Size([2, 19])

In [81]:
result = []
for korean in koreans:
  tokens = tokenizer.convert_ids_to_tokens(korean)
  # 각 korean 시퀀스의 토큰 정수 인덱스 값을 텍스트 토큰으로 변환
  text = tokenizer.convert_tokens_to_string(tokens)
  # 변환된 토큰을 문자열로 합쳐주기
  result.append(text)
result
# 결과로는 변환되고 합쳐진 문자열이 출력

['<pad> 적응 체계를 잘 개발해서 요즘에 기계 통역 시스템은 아무도 도와줄 수 있을 거예요.</s>',
 '<pad> 이 시스템은 한 하루 한 번 파운데이션을 사용해서 만든 시스템입니다.</s><pad>']

In [85]:
from torch.utils.data import DataLoader
test_dataloader = DataLoader(
    tokenized_dataset['test'], batch_size = 32, collate_fn = data_collator
)

In [86]:
test_dataloader_iter = iter(test_dataloader)

In [88]:
test_batch = next(test_dataloader_iter)

In [89]:
test_batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [90]:
test_input = {}
for key in ('input_ids', 'attention_mask'):
  test_input[key] = test_batch[key]
  # 모델에 입력으로 넣을 'input_ids', 'attention_mask'만 추려내기

In [92]:
koreans = model.generate(
    **test_input,
    max_length = max_token_length,
    num_beams = 5
)

In [93]:
labels = np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
# test_batch.labels에서 -100으로 패딩된 값드을 loss에서 계산하지 않기 위해 변환해줌

In [94]:
eng_sentence = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens = True)[10:20]

In [95]:
pred = tokenizer.batch_decode(koreans, skip_special_tokens = True)[10:20]

In [96]:
answer = tokenizer.batch_decode(labels, skip_special_tokens = True)[10:20]

In [98]:
for z in zip(eng_sentence, pred, answer):
  print('eng_sentence :', z[0])
  print('pred :', z[1])
  print('asnwer :', z[2])
  print('\n')

eng_sentence : Isn't that the parking lot behind the building?
pred : 건물 옆에 있는 주차장이 그런 거 아니야?
asnwer : 건물 뒤편에 있는 주차장은 아닌가요?


eng_sentence : Your hair color has come out so well, do you like it?
pred : 머리 색깔이 너무 잘 나오던데, 마음에 드세요?
asnwer : 고객님 염색이 너무 잘 나온 것 같은데 고객님은 마음에 드시나요?


eng_sentence : I did an internship, as well as an in-school investment club.
pred : 제가 인턴도 하고 학교 보험 동아리도 했었어.
asnwer : 교내 투자 동아리와 금융권 인턴십을 한 적이 있습니다.


eng_sentence : I couldn't find it either so I went after looking it up on a map app. It shows up immediately in the app.
pred : 나도 못 찾았어. 지도 앱을 찾아보고 왔어. 앱에 바로 올라오네요.
asnwer : 저도 못 찾았는데 지도 앱을 보고 다녀왔어요. 앱을 켜면 바로 나오더라고요.


eng_sentence : I think you made a prank call, so I'm hanging up now without responding.
pred : 제가 전화한 것 같은데, 연락을 안 해줘.
asnwer : 장난 전화를 거신 것 같은데 더 대응 없이 끊도록 하겠습니다.


eng_sentence : Is the big data analysis by age from last month completed?
pred : 저번 달에 엄청난 데이터 분석은 다 완료되었나요?
asnwer : 저번 달 빅데이터 분석은 연령별로 다 완료되었나요?


eng_sentence : The salad comes out wi